# Sudoku Solver
This notebook includes 3 implementations of the Sudoku solver code from 3 different input soruce
1. Direct input through the notebook 
2. Text file from [dimitri](https://github.com/dimitri/sudoku/blob/master/sudoku.txt)
3. CSV file from [kaggle](https://www.kaggle.com/datasets/rohanrao/sudoku) 

## Inputting the sudoku to be solved
- Feature to be added -> Make it optional to accept the sudoku to be solved through a txt file in the right format
- Make exceptions possible

In [205]:
# Defining the Sudoku to be solved
to_solve =[[5 ,3 ,0, 0, 7, 0, 0, 0, 0],
        [6 ,0 ,0 ,1 ,9 ,5 ,0 ,0 ,0],
        [0 ,9 ,8 ,0 ,0 ,0 ,0 ,6 ,0],
        [8 ,0 ,0 ,0 ,6 ,0 ,0 ,0 ,3],
        [4 ,0 ,0 ,8 ,0 ,3 ,0 ,0 ,1],
        [7 ,0 ,0 ,0 ,2 ,0 ,0 ,0 ,6],
        [0 ,6 ,0 ,0 ,0 ,0 ,2 ,8 ,0],
        [0 ,0 ,0 ,4 ,1 ,9 ,0 ,0 ,5],
        [0 ,0 ,0 ,0 ,8 ,0 ,0 ,7 ,9]]

#### Function to take input form a file and save it in a matrix

In [133]:
# Taking input from a text file for more tests
with open("test.txt") as f:                
    lines = f.readlines()

def MatrixMaker(lines,index = 1):
    to_solve = []
    for i, line in enumerate(lines):
        if line.split(" ")[0] == "Grid":
            if int(line.split(" ")[1]) == index:
                for j in range(i+1,i+10):
                    to_solve.append([])
                    for char in lines[j][:9]:
                        to_solve[-1].append(int(char))
    return to_solve

        

### Code to take input from the kaggle csv file

In [39]:
import pandas as pd
df_sudoku = pd.read_csv("/home/pika/2nd\Brain\Attachments/sudoku.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/home/pika/2nd\\Brain\\Attachments/sudoku.csv'

# Solver 

The program will be using backtracking algorithm to solve the sudoku

### Algorithm
1. Find the first empty square (Positions filled with zeroes are regarded as empty)
2. Now for that sqaure find the approprioate number by naively trying all possibilities from 1-9, checking the horizontal and vertical to verify
3. If no number from 1-9 fits the square, backtrack to the last square that was filled and resume trying with other numbers, untill the cell can be filled 
4. Continue this process till the no empty cell can be found

#### Function to find the first empty sqaure in the sudoku

In [24]:
# Function that finds the first empty square after a given cell
def FindEmpty(sudoku):
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0:
                return i,j
        
    return False

#### Function that varifies if a number is suitable for a location given the matrix

In [25]:
# Function that verifies if a given number can be used in the given cell position 
def Verify(sudoku, row,col,n):

    # Checking the row for the number
    for i in range(9):
        if sudoku[row][i] == n or sudoku[i][col] == n:
            return False
    
    # Checking the 3x3 grid for the number
    start_row = (row // 3) * 3
    start_col = (col // 3) * 3
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if sudoku[i][j] == n:
                return False
    
    return True

### Main Sudoku solver function

In [26]:
# Function that solves the Sudoku using Backtracking
def Solve(sudoku):

    if not FindEmpty(sudoku):
        return True
    
    curx,cury = FindEmpty(sudoku)        
    
    for i in range(1,10):
        if Verify(sudoku, curx,cury,i):
            sudoku[curx][cury] = i
            if Solve(sudoku):
                return True
            sudoku[curx][cury] = 0

    return False

### 1. Input taken directly through the notebook

In [211]:
# Printing the Solution
if Solve(to_solve):
    for row in to_solve:
        print(row)

[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


### 2. Input taken form the txt file

In [165]:
# Printing the result into a file "Output.txt"
with open('result_of_test.txt', 'w') as file:
   for i in range(1,51):
      file.write("Grid ")
      file.write(str(i))
      file.write('\n')
      file.write("-------------------------------------\n")
      see = MatrixMaker(lines,i)
      Solve(see)
      for rows in see:
         file.write("| ")
         for item in rows:
            file.write(str(item))
            file.write(" | ")
         file.write('\n')
         file.write("-------------------------------------\n")
      file.write('\n')

### 3. Input taken from the kaggle csv file

In [36]:
# Cutting the df , so that RAM is saved
limit = 500
df = df_sudoku.head(limit)

solution = []

# Converting the input from kaggle into a matrix , so that it can be solved
def MakeKaggleMatrix(i):
    to_solve = [[],[],[],[],[],[],[],[],[]]
    for j,num in enumerate(df['puzzle'][i]):
        to_solve[j//9].append(int(num))
    return to_solve

# Solving the sudoku and saving that in the same format as kaggle in solutions list
for i in range(limit):
    case = MakeKaggleMatrix(i)
    Solve(case)
    solution.append(0)
    counter = 80
    for rows in case: 
        for item in rows:
            solution[-1] += int(item)* 10**counter
            counter -= 1 

#### Printing the results of the Kaggle in a separate file

In [37]:
with open('result_of_kaggle.txt', 'w') as file:
    for i in range(limit):
        file.write(str(solution[i]))
        file.write('  ->  ')
        if solution[i] == int(df['solution'][i]):
            file.write("CORRECT")
        else:
            file.write("WRONG")
        file.write('\n')